# MongoDB Database

This notebook showcases an agent designed to interact with and query a `MongoDB` database using PyMongo. 
The agent is similar to [SQL Database](https://python.langchain.com/docs/integrations/toolkits/sql_database).

As this agent is in development, currently it only supports one database per instance `MongoDatabase` class using URI. Additionally, all answers may not be correct, and it is not guaranteed that the agent won't perform destructive commands on your database (or in general) given certain questions.

## Initialization

In [1]:
from langchain.agents import create_mongo_agent
from langchain.agents.agent_toolkits import MongoDatabaseToolkit
from langchain.agents.agent_types import AgentType
from langchain.llms.openai import OpenAI
from langchain.utilities.mongo_database import MongoDatabase

db = MongoDatabase.from_uri("mongodb://localhost:27017/my_db")
db._client["my_db"]["my_collection"].insert_many(
    [
        {"text": "Hello, world!", "language": "en"},
        {"text": "Bonjour, monde!", "language": "fr"},
        {"text": "Hola, mundo!", "language": "es"},
        {"text": "Hallo, Welt!", "language": "de"},
        {"text": "Ciao, mondo!", "language": "it"},
        {"text": "Olá, mundo!", "language": "pt"},
        {"text": "Привет, мир!", "language": "ru"},
        {"text": "你好，世界！", "language": "zh"},
        {"text": "こんにちは世界！", "language": "ja"},
        {"text": "안녕, 세상아!", "language": "ko"},
    ]
)
# insert more documents if you would like
toolkit = MongoDatabaseToolkit(db=db, llm=OpenAI(temperature=0))

## Using `ZERO_SHOT_REACT_DESCRIPTION`

This shows how to initialize the agent using the `ZERO_SHOT_REACT_DESCRIPTION` agent type.

In [2]:
agent_executor = create_mongo_agent(
    llm=OpenAI(temperature=0),
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

## Using OpenAI Functions

This shows how to initialize the agent using the `OPENAI_FUNCTIONS` agent type. Note that this is an alternative to the above.

In [ ]:
# agent_executor = create_mongo_agent(
#     llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
#     toolkit=toolkit,
#     verbose=True,
#     agent_type=AgentType.OPENAI_FUNCTIONS
# )

## Example: querying documents

In [3]:
agent_executor.run("Find hello world in french")



> Entering new AgentExecutor chain...
Action: mongo_db_list
Action Input: 
Observation: my_collection
Thought: I should check the schema of my_collection
Action: mongo_db_schema
Action Input: my_collection
Observation: Collection Name: my_collection

3 sample documents from my_collection:
{'_id': ObjectId('65650d24fd0c16012a7ed210'),
 'language': 'en',
 'text': 'Hello, world!'}
{'_id': ObjectId('65650d24fd0c16012a7ed211'),
 'language': 'fr',
 'text': 'Bonjour, monde!'}
{'_id': ObjectId('65650d24fd0c16012a7ed212'),
 'language': 'es',
 'text': 'Hola, mundo!'}

Thought: I should query the documents with language field equal to 'fr'
Action: mongo_db_query
Action Input: db.my_collection.find({'language': 'fr'}).limit(10)
Observation: Result:
{'_id': ObjectId('65650d24fd0c16012a7ed211'),
 'language': 'fr',
 'text': 'Bonjour, monde!'}
{'_id': ObjectId('65650d2bfd0c16012a7ed21c'),
 'language': 'fr',
 'text': 'Bonjour, monde!'}
{'_id': ObjectId('65650f9af8d68bbbc66c0c2e'),
 'language': 'fr',


'Bonjour, monde!'